<a href="https://colab.research.google.com/github/binnini/DM-project-team16/blob/main/TwitterPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# local에 padnas, numpy, matplotlib 설치필요
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from pyspark.sql.types import StructField, StructType, StringType, DoubleType, FloatType, IntegerType
from pyspark.sql.functions import col

# local에 pyspark 설치필요
!pip install pyspark
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

# # Spark 세션 생성
spark = SparkSession.builder \
    .appName("RS_AmazonXTwitter") \
    .getOrCreate()

In [2]:
# # Step 0: Get Kaggle API Key at Kaggle

# # Step 1: Set Kaggle API Key
# from google.colab import files
# files.upload()  # kaggle.json file upload

# # Step 2: API Key copy authorization
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

# Step 3: Kaggle CLI 설치
!pip install kaggle

# Step 4: 데이터셋 다운로드
!kaggle datasets download -d kazanova/sentiment140
# !kaggle datasets download -d snap/amazon-fine-food-reviews
# !kaggle datasets download -d karkavelrajaj/amazon-sales-dataset
!kaggle datasets download -d asaniczka/amazon-products-dataset-2023-1-4m-products
# !kaggle datasets download -d mexwell/amazon-reviews-multi


# Step 5: 데이터 압축 해제
!unzip sentiment140.zip
# !unzip amazon-fine-food-reviews.zip
# !unzip amazon-sales-dataset.zip
!unzip amazon-products-dataset-2023-1-4m-products.zip
# !unzip amazon-reviews-multi.zip

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 90% 73.0M/80.9M [00:00<00:00, 128MB/s]
100% 80.9M/80.9M [00:00<00:00, 122MB/s]
Dataset URL: https://www.kaggle.com/datasets/asaniczka/amazon-products-dataset-2023-1-4m-products
License(s): ODC Attribution License (ODC-By)
 96% 95.0M/99.2M [00:00<00:00, 139MB/s]
100% 99.2M/99.2M [00:00<00:00, 134MB/s]
Archive:  sentiment140.zip
  inflating: training.1600000.processed.noemoticon.csv  
Archive:  amazon-products-dataset-2023-1-4m-products.zip
  inflating: amazon_categories.csv   
  inflating: amazon_products.csv     


# 1. Twitter data

In [138]:
# file path
input_file = "/content/training.1600000.processed.noemoticon.csv"

# raw data
tw_raw_df = spark.read.format("csv").option("header", "false").load(input_file)

# set schema
tw_schema = StructType([
    StructField("target", DoubleType(), True),   # Sentiment (0/1)
    StructField("id", DoubleType(), True),       # Tweet ID
    StructField("date", StringType(), True),     # Date
    StructField("query", StringType(), True),    # Query (if applicable)
    StructField("author", StringType(), True),   # Author/User
    StructField("content", StringType(), True),  # Tweet content
])

tw_formatted_df = tw_raw_df.toDF(*[field.name for field in tw_schema.fields])

# select needed column
tw_formatted_df = tw_formatted_df.select("id", "author", "content")

# check data
tw_formatted_df.printSchema()
tw_formatted_df.show(5)

authors = tw_formatted_df.select("author").distinct()

print("total count : ",tw_formatted_df.count())
print("authors count : ",authors.count())

root
 |-- id: string (nullable = true)
 |-- author: string (nullable = true)
 |-- content: string (nullable = true)

+----------+---------------+--------------------+
|        id|         author|             content|
+----------+---------------+--------------------+
|1467810369|_TheSpecialOne_|@switchfoot http:...|
|1467810672|  scotthamilton|is upset that he ...|
|1467810917|       mattycus|@Kenichan I dived...|
|1467811184|        ElleCTF|my whole body fee...|
|1467811193|         Karoli|@nationwideclass ...|
+----------+---------------+--------------------+
only showing top 5 rows

total count :  1600000


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

**1.0. Preprocessing**

**1-1. Categorized tweet by dictionary**

In [ ]:
# # from pyspark.sql.functions import col, lower, when, size, array, expr
# from pyspark.sql.functions import explode, split, col, lower, count, desc, max

# # 입력 파일 경로
# input_file = "/content/training.1600000.processed.noemoticon.csv"

# # 데이터 스키마 정의
# tw_schema = StructType([
#     StructField("target", DoubleType(), True),
#     StructField("id", DoubleType(), True),
#     StructField("date", StringType(), True),
#     StructField("query", StringType(), True),
#     StructField("author", StringType(), True),
#     StructField("content", StringType(), True)
# ])

# # 데이터 로드 및 스키마 적용
# tw_raw_df = spark.read.format("csv").option("header", "false").load(input_file)
# tw_formatted_df = tw_raw_df.toDF(*[field.name for field in tw_schema.fields])

# # 데이터 샘플링
# categorized_df = tw_formatted_df.select("content").sample(fraction=0.01, seed=42)

# # 카테고리 키워드 딕셔너리 정의
# category_keywords = {
#     "jewelry": ["ring", "necklace", "bracelet", "earrings", "pendant", "brooch", "chain", "charm", "gold", "silver"],
#     "watch": ["watch", "timepiece", "chronograph", "quartz", "analog", "digital", "wristwatch", "strap", "dial", "bezel"],
#     "drugstore": ["pharmacy", "medicine", "drugs", "prescription", "pill", "vitamin", "tablet", "pharmacist", "aspirin", "capsule"],
#     "pet_products": ["pet", "dog", "cat", "fish", "bird", "hamster", "leash", "kennel", "collar", "aquarium"],
#     "automotive": ["car", "vehicle", "engine", "automobile", "tire", "wheel", "battery", "transmission", "brake", "sedan"],
#     "shoes": ["shoe", "sneakers", "boots", "heels", "sandals", "loafers", "oxfords", "slippers", "cleats", "flats"],
#     "beauty": ["beauty", "makeup", "cosmetics", "lipstick", "skincare", "mascara", "foundation", "blush", "palette", "concealer"],
#     "office_product": ["office", "stationery", "desk", "printer", "paper", "pen", "notebook", "folder", "calculator", "stapler"],
#     "digital_ebook_purchase": ["ebook", "kindle", "digital book", "pdf", "epub", "audiobook", "download", "reader", "publication", "novel"],
#     "lawn_and_garden": ["garden", "lawn", "yard", "mower", "fertilizer", "plant", "seeds", "shovel", "hedge", "sprinkler"],
#     "kitchen": ["kitchen", "cookware", "appliance", "utensils", "cutlery", "oven", "stove", "pan", "mixer", "microwave"],
#     "apparel": ["clothing", "apparel", "dress", "shirt", "jacket", "jeans", "skirt", "sweater", "suit", "blouse"],
#     "industrial_supplies": ["industrial", "tools", "hardware", "equipment", "machinery", "fastener", "drill", "screwdriver", "wrench", "saw"],
#     "wireless": ["wireless", "network", "wifi", "router", "modem", "signal", "antenna", "hotspot", "connectivity", "bluetooth"],
#     "digital_video_download": ["video", "movie", "streaming", "download", "hd", "4k", "bluray", "rental", "cinema", "series"],
#     "grocery": ["food", "grocery", "snacks", "vegetables", "fruits", "bread", "milk", "meat", "cereal", "beverage"],
#     "personal_care_appliances": ["care", "appliance", "shaver", "toothbrush", "trimmer", "hairdryer", "curler", "straightener", "massager", "razor"],
#     "furniture": ["furniture", "sofa", "table", "chair", "bed", "dresser", "cabinet", "shelf", "couch", "armchair"],
#     "baby_product": ["baby", "infant", "crib", "diaper", "stroller", "bottle", "pacifier", "highchair", "blanket", "bib"],
#     "electronics": ["electronics", "device", "gadget", "charger", "headphones", "camera", "tablet", "speaker", "remote", "monitor"],
#     "sports": ["sports", "equipment", "fitness", "workout", "gear", "tennis", "soccer", "basketball", "yoga", "cycling"],
#     "home": ["home", "house", "living", "decor", "curtains", "lighting", "rug", "pillow", "mirror", "wall art"],
#     "pc": ["pc", "laptop", "computer", "keyboard", "mouse", "monitor", "cpu", "notebook", "desktop", "hardware"],
#     "luggage": ["luggage", "bag", "suitcase", "backpack", "duffle", "trolley", "pouch", "wallet", "briefcase", "travel"],
#     "home_improvement": ["improvement", "renovation", "diy", "paint", "tiles", "fixtures", "drill", "ladder", "flooring", "cement"],
#     "toy": ["toy", "game", "doll", "lego", "puzzle", "blocks", "teddy", "action figure", "boardgame", "train"],
#     "camera": ["camera", "dslr", "photo", "lens", "tripod", "shutter", "focus", "mirrorless", "film", "flash"],
#     "book": ["book", "novel", "read", "fiction", "non-fiction", "magazine", "literature", "paperback", "hardcover", "author"],
#     "video_games": ["game", "console", "xbox", "playstation", "nintendo", "pc gaming", "joystick", "controller", "esports", "arcade"],
#     "musical_instruments": ["instrument", "guitar", "piano", "violin", "drums", "flute", "keyboard", "trumpet", "saxophone", "harp"]
# }


# # 카테고리 키워드 데이터프레임 생성
# categories = [(category, keyword) for category, keywords in category_keywords.items() for keyword in keywords]
# category_df = spark.createDataFrame(categories, ["category", "keyword"])

# # 트윗 내용을 단어로 분리
# words_df = categorized_df.withColumn("word", explode(split(lower(col("content")), "\\W+")))

# # 키워드와 트윗 단어 매칭
# matched_df = words_df.join(
#     category_df,
#     words_df.word == category_df.keyword,
#     "inner"
# ).groupBy("content", "category").agg(count("keyword").alias("match_count"))

# categorized_df_with_alias = categorized_df.alias("categorized")
# matched_df_with_alias = matched_df.alias("matched")

# all_matched_df = categorized_df_with_alias.join(
#     matched_df_with_alias,
#     col("categorized.content") == col("matched.content"),
#     "left_outer"
# ).select(
#     col("categorized.content").alias("content"),  # categorized_df의 content 열 선택
#     coalesce(col("matched.category"), lit("others")).alias("category"),  # 매칭되지 않은 경우 'others'로 설정
#     coalesce(col("matched.match_count"), lit(0)).alias("match_count")   # 매칭되지 않은 경우 0으로 설정
# )

# # 최종 데이터 출력
# all_matched_df.show(truncate=False)

In [ ]:
# # 1. 카테고리별 개수 집계
# category_counts = all_matched_df.groupBy("category").count()

# # 2. 전체 데이터 개수 계산
# total_count = all_matched_df.count()

# # 3. 비율 계산 (새로운 컬럼 추가)
# category_ratios = category_counts.withColumn(
#     "ratio", (col("count") / total_count) * 100
# )

# # 4. Spark DataFrame을 Pandas DataFrame으로 변환
# category_ratios_pd = category_ratios.toPandas()

# # 5. 시각화: 눈금 조정
# plt.figure(figsize=(10, 6))
# plt.bar(category_ratios_pd['category'], category_ratios_pd['ratio'], alpha=0.7)
# plt.title('Category Ratio Distribution', fontsize=16)
# plt.xlabel('Category', fontsize=14)
# plt.ylabel('Ratio (%)', fontsize=14)

# # Y축 눈금과 범위 설정 (예: 최대값을 50%로 제한)
# plt.ylim(0, 50)  # Y축 범위를 0~50으로 설정
# plt.yticks(range(0, 51, 10))  # 0부터 50까지 10 단위 눈금

# # X축 라벨 설정
# plt.xticks(rotation=45)
# plt.grid(axis='y', linestyle='--', alpha=0.7)
# plt.tight_layout()

# # 6. 그래프 출력
# plt.show()

In [ ]:
# specific_category = "automotive"
# filtered_tweets = all_matched_df.filter(col("category") == specific_category)
# filtered_tweets.show(truncate=False)

**1-2. Categorized tweet by TF-IDF**

# 2. Amazon Data

**2-1. asaniczka/amazon-products-dataset-2023-1-4m-products** :

*   https://www.kaggle.com/datasets/asaniczka/amazon-products-dataset-2023-1-4m-products
*   Category is too much.




In [178]:
# file path
product_path = "/content/amazon_products.csv"
category_path = "/content/amazon_categories.csv"

# raw data
az_raw_df = spark.read.format("csv") \
    .option("header", "true") \
    .load(product_path)

cat_df = spark.read.format("csv") \
    .option("header", "true") \
    .load(category_path)

az_raw_df.printSchema()
cat_df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- title: string (nullable = true)
 |-- imgUrl: string (nullable = true)
 |-- productURL: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- reviews: string (nullable = true)
 |-- price: string (nullable = true)
 |-- listPrice: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- isBestSeller: string (nullable = true)
 |-- boughtInLastMonth: string (nullable = true)

root
 |-- id: string (nullable = true)
 |-- category_name: string (nullable = true)



데이터 정제

In [179]:
filtered_df = az_raw_df.filter(
    (col("title").cast("int").isNull())
)

filtered_df = az_raw_df.filter(
    (col("category_id").cast("int").isNotNull()) &  # 정수로 변환 가능
    (col("category_id").cast("int") >= 1) &        # 최소값 1
    (col("category_id").cast("int") <= 270)        # 최대값 270
)

filtered_df = filtered_df.filter(
    (col("stars").cast("float").isNotNull()) &  # 부동소수점으로 변환 가능
    (col("stars").cast("float") >= 0) &        # 최소값 0
    (col("stars").cast("float") <= 5)        # 최대값 5
)

filtered_df = filtered_df.filter(
    (col("reviews").cast("int").isNotNull()) &  # 부동소수점으로 변환 가능
    (col("reviews").cast("int") >= 0) &        # 최소값 0
    (col("reviews").cast("int") <= 100)        # 최대값 100 (임의로 정함)
)

cleaned_df = filtered_df\
  .withColumn("category_id", col("category_id").cast("int"))\
  .withColumn("stars", col("stars").cast("float"))\
  .withColumn("reviews", col("reviews").cast("int"))

# 결과 확인
# cleaned_df.show()
cleaned_df.select("asin","title","stars","reviews","category_id").describe().show()

+-------+--------------------+--------------------+------------------+------------------+-----------------+
|summary|                asin|               title|             stars|           reviews|      category_id|
+-------+--------------------+--------------------+------------------+------------------+-----------------+
|  count|             1228601|             1228601|           1228601|           1228601|          1228601|
|   mean|1.7932850190857334E9|3.846153846153846...|3.9480175339364014| 3.310014398490641|123.8509182395261|
| stddev|1.8480468674133875E9|            Infinity|1.4063996948736122|13.033869416810788|72.83617705402432|
|    min|          0007268149| AIYUENCICI Funny...|               0.0|                 0|                1|
|    max|          BT00CTP93I|📬 United States ...|               5.0|               100|              270|
+-------+--------------------+--------------------+------------------+------------------+-----------------+



In [176]:
# from pyspark.sql.functions import col, length

# # title 열의 길이 계산
# title_with_lengths = cleaned_df.withColumn("title_length", length(col("title")))

# # 최소 길이의 요소 찾기
# min_length_row = title_with_lengths.orderBy(col("title_length").asc()).first()

# # 최대 길이의 요소 찾기
# max_length_row = title_with_lengths.orderBy(col("title_length").desc()).first()

# # 결과 출력
# print(f"최소 길이 요소: {min_length_row['title']} (길이: {min_length_row['title_length']})")
# print(f"최대 길이 요소: {max_length_row['title']} (길이: {max_length_row['title_length']})")

각 열의 고유한 값 확인

In [165]:
# asin의 고유 값과 비율 계산
asin_distribution = cleaned_df.groupBy("asin").count()

# 전체 데이터 수 계산
total_count = cleaned_df.count()

# 비율 계산 추가
asin_distribution = asin_distribution.withColumn("percentage", (col("count") / total_count) * 100)

# percentage 기준으로 내림차순 정렬
asin_distribution = asin_distribution.orderBy(col("percentage").desc())

# 결과 출력
asin_distribution.show()

unique_count = cleaned_df.select("asin").distinct().count()
print(f"asin의 고유 값 개수: {unique_count}")

+----------+-----+--------------------+
|      asin|count|          percentage|
+----------+-----+--------------------+
|B09R35YK22|    1|8.139338971724751E-5|
|B08XMYH4L2|    1|8.139338971724751E-5|
|B0BSYVWQMN|    1|8.139338971724751E-5|
|B07G23DCNL|    1|8.139338971724751E-5|
|B07JJBDP68|    1|8.139338971724751E-5|
|B084MHG7GG|    1|8.139338971724751E-5|
|B06XCQSPKL|    1|8.139338971724751E-5|
|B0C4NR75R2|    1|8.139338971724751E-5|
|B07Q34YLTW|    1|8.139338971724751E-5|
|B01L8JJ57U|    1|8.139338971724751E-5|
|B00HG4ER9I|    1|8.139338971724751E-5|
|B07VTSD48H|    1|8.139338971724751E-5|
|B071P2M86D|    1|8.139338971724751E-5|
|B01K5YKEA6|    1|8.139338971724751E-5|
|B09RSCJTVL|    1|8.139338971724751E-5|
|B074XY7MTG|    1|8.139338971724751E-5|
|B0BBTZ9F76|    1|8.139338971724751E-5|
|B087CHW9B7|    1|8.139338971724751E-5|
|B0BK11PDLL|    1|8.139338971724751E-5|
|B07L6LH5N9|    1|8.139338971724751E-5|
+----------+-----+--------------------+
only showing top 20 rows

asin의 고유 값 개수:

In [171]:
# title의 고유 값과 비율 계산
title_distribution = cleaned_df.groupBy("title").count()

# 전체 데이터 수 계산
total_count = cleaned_df.count()

# 비율 계산 추가
title_distribution = title_distribution.withColumn("percentage", (col("count") / total_count) * 100)

# percentage 기준으로 내림차순 정렬
title_distribution = title_distribution.orderBy(col("percentage").desc())

# 결과 출력
title_distribution.show()

unique_count = cleaned_df.select("title").distinct().count()
print(f"title의 고유 값 개수: {unique_count}")

+--------------------+-----+--------------------+
|               title|count|          percentage|
+--------------------+-----+--------------------+
|         mens Modern|   85|0.006918438125966038|
|       Men's Sneaker|   84| 0.00683704473624879|
|Men's Ultraboost ...|   83|0.006755651346531544|
|Sleep and Play PJ...|   77|0.006267291008228...|
|        mens Sneaker|   69|0.005616143890490078|
|Amazon.com Gift C...|   64|0.005209176941903841|
|Burt's Bees Baby ...|   62|0.005046390162469345|
|BCB Controller Cu...|   60| 0.00488360338303485|
|Men's Running Sho...|   54|0.004395243044731365|
|     Women's Sneaker|   53|0.004313849655014118|
|Men's Newport H2 ...|   52| 0.00423245626529687|
|Customized lenses...|   43|0.003499915757841643|
|Multipack 2-Piece...|   39|0.003174342198972...|
|Kids' Plastic Tim...|   36| 0.00293016202982091|
|Girl's Limited Ed...|   35|0.002848768640103...|
|Unisex-Child Sneaker|   33|0.002685981860669168|
|Princess Kids' Pl...|   31|0.002523195081234...|


In [153]:
# category_id의 고유 값과 비율 계산
category_id_distribution = cleaned_df.groupBy("category_id").count()

# 전체 데이터 수 계산
total_count = cleaned_df.count()

# 비율 계산 추가
category_id_distribution = category_id_distribution.withColumn("percentage", (col("count") / total_count) * 100)

# percentage 기준으로 내림차순 정렬
category_id_distribution = category_id_distribution.orderBy(col("percentage").desc())

# 결과 출력
category_id_distribution.show()

unique_count = cleaned_df.select("category_id").distinct().count()
print(f"category_id의 고유 값 개수: {unique_count}")

+-----------+-----+------------------+
|category_id|count|        percentage|
+-----------+-----+------------------+
|         91|28613|2.0845976696627035|
|         84|24646|1.7955822236922725|
|        270|20531|1.4957842503702852|
|        114|19657|1.4321090550644728|
|        118|18968|1.3819120189481062|
|         95|18431|1.3427889298414457|
|        110|18215|1.3270522683013366|
|        112|17573|1.2802794131682342|
|        116|17371|1.2655627204316506|
|        123|16882|1.2299366672227925|
|        105|16085| 1.171871300336371|
|        122|15605|1.1369009413583506|
|        173|14644|1.0668873684877724|
|        120|13868|1.0103519548066395|
|        113|12817|0.9337814396276822|
|         97|10935|0.7966684904680273|
|         88|10800|0.7868330770054591|
|         51| 9481|0.6907374447304404|
|        107| 9383|0.6835976631057613|
|         72| 9249|0.6738351045577305|
+-----------+-----+------------------+
only showing top 20 rows

category_id의 고유 값 개수: 248


In [151]:
# stars의 고유 값과 비율 계산
stars_distribution = cleaned_df.groupBy("stars").count()

# 전체 데이터 수 계산
total_count = cleaned_df.count()

# 비율 계산 추가
stars_distribution = stars_distribution.withColumn("percentage", (col("count") / total_count) * 100)

# percentage 기준으로 내림차순 정렬
stars_distribution = stars_distribution.orderBy(col("percentage").desc())

# 결과 출력
stars_distribution.show()

unique_count = cleaned_df.select("stars").distinct().count()
print(f"stars의 고유 값 개수: {unique_count}")

+-----+------+------------------+
|stars| count|        percentage|
+-----+------+------------------+
|  4.6|168423|12.931026540301826|
|  4.5|161680|12.413318674029076|
|  4.7|140971|10.823342075683778|
|  4.4|132071|10.140026042786332|
|  0.0|111314| 8.546364144488326|
|  4.3|103687| 7.960785337419922|
|  5.0| 86749|6.6603351166090325|
|  4.8| 75496|  5.79636260894668|
|  4.2| 74754| 5.739394013844444|
|  4.1| 53258| 4.088993851691246|
|  4.0| 46990|3.6077550995338097|
|  3.9| 28894| 2.218397017363905|
|  3.8| 21835|1.6764275930691792|
|  4.9| 20217| 1.552202273830071|
|  3.7| 15074|1.1573377393141657|
|  3.6| 11489|0.8820918990964873|
|  3.5| 10133|0.7779821754325621|
|  3.0|  6733|0.5169400954492688|
|  3.4|  6729|0.5166329871198767|
|  3.3|  5258|0.4036938989859283|
+-----+------+------------------+
only showing top 20 rows

stars의 고유 값 개수: 41


In [161]:
# reviews의 고유 값과 비율 계산
reviews_distribution = cleaned_df.groupBy("reviews").count()

# 전체 데이터 수 계산
total_count = cleaned_df.count()

# 비율 계산 추가
reviews_distribution = reviews_distribution.withColumn("percentage", (col("count") / total_count) * 100)

# percentage 기준으로 내림차순 정렬
reviews_distribution = reviews_distribution.orderBy(col("percentage").desc())

# 결과 출력
reviews_distribution.show()

unique_count = cleaned_df.select("reviews").distinct().count()
print(f"reviews의 고유 값 개수: {unique_count}")

+-------+-------+-------------------+
|reviews|  count|         percentage|
+-------+-------+-------------------+
|      0|1085850|   88.3810122244732|
|      1|  11200| 0.9116059648331721|
|      2|   8241| 0.6707629246598367|
|      3|   6442| 0.5243362165585085|
|      4|   5364| 0.4365941424433156|
|      5|   4528|0.36854926863969667|
|      6|   4103| 0.3339570780098665|
|      7|   3633| 0.2957021848427602|
|      8|   3276| 0.2666447447137028|
|      9|   3063| 0.2493079527039291|
|     10|   2849|0.23188976730443814|
|     11|   2630|0.21406461495636092|
|     12|   2457| 0.1999835585352771|
|     13|   2399|0.19526274193167678|
|     14|   2271|0.18484438804786907|
|     16|   2074|0.16880989027357132|
|     15|   2062|0.16783316959696437|
|     17|   2056|0.16734480925866085|
|     18|   1911|0.15554276774965997|
|     19|   1843|0.15000801724888715|
+-------+-------+-------------------+
only showing top 20 rows

reviews의 고유 값 개수: 101


데이터 스키마 적용

In [180]:
az_df = cleaned_df.select("asin", "title", "stars","reviews","category_id")

az_df = az_df \
    .withColumn("stars", col("stars").cast(FloatType())) \
    .withColumn("reviews", col("reviews").cast(IntegerType())) \
    .withColumn("category_id", col("category_id").cast(IntegerType()))

az_schema = StructType([
    StructField("asin", StringType(), False),         # asin: StringType, nullable=False
    StructField("title", StringType(), False),        # title: StringType, nullable=False
    StructField("stars", FloatType(), True),          # stars: FloatType, nullable=True
    StructField("reviews", IntegerType(), True),      # reviews: IntegerType, nullable=True
    StructField("category_id", IntegerType(), False),  # category_id: IntegerType, nullable=False
])

az_formatted_df = spark.createDataFrame(az_df.rdd, schema=az_schema)

# 고유 값 출력
print("정제된 데이터 개수 : ",az_formatted_df.count())

정제된 데이터 개수 :  1228601


In [181]:
cat_df = cat_df \
    .withColumn("id", col("id").cast(IntegerType()))\
    .withColumn("category_name", col("category_name").cast(StringType()))

cat_schema = StructType([
    StructField("id", IntegerType(), False),         # asin: StringType, nullable=False
    StructField("category_name", StringType(), False),        # title: StringType, nullable=False
])

cat_formatted_df = spark.createDataFrame(cat_df.rdd, schema=cat_schema)

cat_formatted_df = cat_formatted_df.fillna({"id": 0, "category_name": "Unknown"})

cat_formatted_df.printSchema()
cat_formatted_df.show()

root
 |-- id: integer (nullable = false)
 |-- category_name: string (nullable = false)

+---+--------------------+
| id|       category_name|
+---+--------------------+
|  1|Beading & Jewelry...|
|  2|   Fabric Decorating|
|  3|Knitting & Croche...|
|  4|Printmaking Supplies|
|  5|Scrapbooking & St...|
|  6|     Sewing Products|
|  7|Craft & Hobby Fabric|
|  8| Needlework Supplies|
|  9|Arts, Crafts & Se...|
| 10|Painting, Drawing...|
| 11|Craft Supplies & ...|
| 12|Gift Wrapping Sup...|
| 13|   Party Decorations|
| 14|Automotive Paint ...|
| 15|Heavy Duty & Comm...|
| 16|Automotive Tires ...|
| 17|Automotive Tools ...|
| 18|Automotive Perfor...|
| 19|            Car Care|
| 20|       Oils & Fluids|
+---+--------------------+
only showing top 20 rows



두 데이터 병합

In [182]:
joined_df = az_formatted_df.join(cat_formatted_df, az_formatted_df.category_id == cat_formatted_df.id, "inner")

joined_df = joined_df.select(
    "asin",
    "title",
    "stars",
    "reviews",
    "category_id",
    "category_name",
)

# 결과 확인
joined_df.printSchema()
joined_df.show()


root
 |-- asin: string (nullable = false)
 |-- title: string (nullable = false)
 |-- stars: float (nullable = true)
 |-- reviews: integer (nullable = true)
 |-- category_id: integer (nullable = false)
 |-- category_name: string (nullable = false)

+----------+--------------------+-----+-------+-----------+--------------------+
|      asin|               title|stars|reviews|category_id|       category_name|
+----------+--------------------+-----+-------+-----------+--------------------+
|B0CBN65GY5|Sasttie Pregnancy...|  5.0|      4|         31|Pregnancy & Mater...|
|B0C3PM7P1N|Frida Fertility E...|  4.7|     97|         31|Pregnancy & Mater...|
|B0CGD489R3|taynoes Pregnancy...|  4.7|     59|         31|Pregnancy & Mater...|
|B08B8Y8ZJT|Natural Cycles Ov...|  4.7|     42|         31|Pregnancy & Mater...|
|B0C4SSDQYR|Momcozy Pregnancy...|  4.3|     53|         31|Pregnancy & Mater...|
|B0C3RWRCQM|PILLANI Nursing P...|  4.5|     29|         31|Pregnancy & Mater...|
|B0C3T4F4LN|New Baby Bo

null 값 존재 여부 확인, null값 0으로 대체

In [183]:
null_counts = joined_df.select([
    sum(when(col(column).isNull(), 1).otherwise(0)).alias(column)
    for column in joined_df.columns
])

null_counts.show()

joined_df = joined_df.fillna({
    "stars": 0,
    "reviews": 0,
})

+----+-----+-----+-------+-----------+-------------+
|asin|title|stars|reviews|category_id|category_name|
+----+-----+-----+-------+-----------+-------------+
|   0|    0|    0|      0|          0|            0|
+----+-----+-----+-------+-----------+-------------+



수치 정보 요약



In [184]:
# Numerical summary
joined_df.describe().show()

+-------+--------------------+--------------------+------------------+------------------+-----------------+--------------------+
|summary|                asin|               title|             stars|           reviews|      category_id|       category_name|
+-------+--------------------+--------------------+------------------+------------------+-----------------+--------------------+
|  count|             1228601|             1228601|           1228601|           1228601|          1228601|             1228601|
|   mean|1.7932850190857334E9|3.846153846153846...|3.9480175339364014| 3.310014398490641|123.8509182395261|                NULL|
| stddev| 1.848046867413387E9|                 NaN|1.4063996948736115|13.033869416810948|72.83617705402303|                NULL|
|    min|          0007268149| AIYUENCICI Funny...|               0.0|                 0|                1|Abrasive & Finish...|
|    max|          BT00CTP93I|📬 United States ...|               5.0|               100|         

In [185]:
# Compute unique values for each column
unique_values = {col: joined_df.select(col).distinct().count() for col in joined_df.columns}

# Print unique values
print("Unique values in each column:")
for col, count in unique_values.items():
    print(f"{col}: {count}")

Unique values in each column:
asin: 1228601
title: 1198610
stars: 41
reviews: 101
category_id: 248
category_name: 248
